In [ ]:
import requests
import numpy as np
import pandas as pd
import tensorflow as tf
from tensorflow.keras.models import load_model
from tensorflow.keras.losses import MeanSquaredError
from sklearn.preprocessing import MinMaxScaler
import joblib
import psycopg2
import time
import ssl
import paho.mqtt.client as mqtt
import json

# === PostgreSQL RDS Database Credentials ===
DB_HOST = "raktrishanan.cjy0uo4meyfo.eu-north-1.rds.amazonaws.com"
DB_NAME = "postgres"
DB_USER = "postgres_srp"
DB_PASSWORD = "RAKTRISHANAN"
DB_PORT = 5432

# === AWS IoT MQTT Setup ===
AWS_IOT_ENDPOINT = "a15spwahok8v1c-ats.iot.eu-north-1.amazonaws.com"
AWS_PORT = 8883
PUBLISH_TOPIC = "serial/data"
CA_PATH = "AmazonRootCA1.pem"
CERT_PATH = "d21a088c2955651c43c59e925ceceaf66fd310f5c29b04f933e78dc30126834e-certificate.pem.crt"
KEY_PATH = "d21a088c2955651c43c59e925ceceaf66fd310f5c29b04f933e78dc30126834e-private.pem.key"

mqtt_client = mqtt.Client()
mqtt_client.tls_set(ca_certs=CA_PATH,
                    certfile=CERT_PATH,
                    keyfile=KEY_PATH,
                    tls_version=ssl.PROTOCOL_TLSv1_2)
mqtt_client.connect(AWS_IOT_ENDPOINT, AWS_PORT)
mqtt_client.loop_start()

def publish_abnormal_activity(details="Anomaly Detected"):
    # MQTT publish (as before)
    mqtt_client.publish(PUBLISH_TOPIC, json.dumps({"message": details}))
    print("📡 Abnormal activity published to AWS IoT")


# === Load Scaler and Model Once ===
scaler = joblib.load("scaler.pkl")
autoencoder = load_model("lstm_autoencoder.h5", custom_objects={"mse": MeanSquaredError()})
sequence_length = 10
batch_size = 10  # every 10 new rows
threshold_percentile = 95
threshold_factor = 2.5
processed_ids = set()  # Track already processed rows

def detect_sudden_changes(data, threshold_factor=2.5):
    std_values = np.std(data, axis=0)
    anomaly_indices = []
    for i in range(1, len(data)):
        deviation = np.abs(data[i] - data[i-1])
        if np.any(deviation > threshold_factor * std_values):
            anomaly_indices.append(i)
    return anomaly_indices

# === Main Loop ===
print("🔄 Starting continuous anomaly monitoring and IoT publishing...")
while True:
    try:
        # Connect to DB
        conn = psycopg2.connect(
            host=DB_HOST,
            database=DB_NAME,
            user=DB_USER,
            password=DB_PASSWORD,
            port=DB_PORT
        )

        # Fetch latest data
        query = """
            SELECT id, ax, ay, az, gx, gy, gz
            FROM sensor_data
            ORDER BY id DESC
            LIMIT 5000
        """
        sensor_data = pd.read_sql(query, conn)
        conn.close()

        # Filter only unprocessed data
        sensor_data = sensor_data[~sensor_data['id'].isin(processed_ids)]

        if len(sensor_data) >= batch_size:
            sensor_data = sensor_data.sort_values(by='id')
            processed_ids.update(sensor_data['id'].tolist())

            sensor_values = sensor_data[['ax', 'ay', 'az', 'gx', 'gy', 'gz']].values
            sensor_values_scaled = scaler.transform(sensor_values)

            X_test = []
            for i in range(len(sensor_values_scaled) - sequence_length):
                X_test.append(sensor_values_scaled[i:i + sequence_length])
            X_test = np.array(X_test)

            if len(X_test) > 0:
                reconstructed = autoencoder.predict(X_test)
                mse = np.mean(np.abs(X_test - reconstructed), axis=(1, 2))
                threshold = np.percentile(mse, threshold_percentile)
                anomalies = mse > threshold

                sudden_change_anomalies = detect_sudden_changes(sensor_values)

                for i, is_anomaly in enumerate(anomalies):
                    batch_status = f"Batch {i+1}:"
                    if is_anomaly or i in sudden_change_anomalies:
                        msg = f"{batch_status} Abnormal Activity Detected"
                        print(f"[!] {msg}")
                        publish_abnormal_activity(msg)
                    else:
                        print(f"[✓] {batch_status} Normal Activity")
            else:
                print("⚠️ Not enough data for sequence generation.")
        else:
            print(f"Waiting for new data... (Currently have {len(sensor_data)} unprocessed rows)")

        time.sleep(5)

    except Exception as e:
        print("❌ Error during processing:", e)
        time.sleep(10)


C:\Users\Trisha\AppData\Local\Temp\ipykernel_3920\2777328261.py:30: DeprecationWarning: Callback API version 1 is deprecated, update to latest version
  mqtt_client = mqtt.Client()


🔄 Starting continuous anomaly monitoring and IoT publishing...


C:\Users\Trisha\AppData\Local\Temp\ipykernel_3920\2777328261.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sensor_data = pd.read_sql(query, conn)
C:\Users\Trisha\Downloads\New folder\ANACONDA\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


156/156 ━━━━━━━━━━━━━━━━━━━━ 3s 11ms/step
[✓] Batch 1: Normal Activity
[✓] Batch 2: Normal Activity
[✓] Batch 3: Normal Activity
[✓] Batch 4: Normal Activity
[✓] Batch 5: Normal Activity
[✓] Batch 6: Normal Activity
[✓] Batch 7: Normal Activity
[✓] Batch 8: Normal Activity
[✓] Batch 9: Normal Activity
[✓] Batch 10: Normal Activity
[✓] Batch 11: Normal Activity
[✓] Batch 12: Normal Activity
[✓] Batch 13: Normal Activity
[✓] Batch 14: Normal Activity
[✓] Batch 15: Normal Activity
[✓] Batch 16: Normal Activity
[✓] Batch 17: Normal Activity
[✓] Batch 18: Normal Activity
[✓] Batch 19: Normal Activity
[✓] Batch 20: Normal Activity
[✓] Batch 21: Normal Activity
[✓] Batch 22: Normal Activity
[✓] Batch 23: Normal Activity
[✓] Batch 24: Normal Activity
[✓] Batch 25: Normal Activity
[✓] Batch 26: Normal Activity
[✓] Batch 27: Normal Activity
[✓] Batch 28: Normal Activity
[✓] Batch 29: Normal Activity
[✓] Batch 30: Normal Activity
[✓] Batch 31: Normal Activity
[✓] Batch 32: Normal Activity
[✓] Bat

C:\Users\Trisha\AppData\Local\Temp\ipykernel_3920\2777328261.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sensor_data = pd.read_sql(query, conn)


...

Waiting for new data... (Currently have 0 unprocessed rows)
Waiting for new data... (Currently have 4 unprocessed rows)
1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 37ms/step


C:\Users\Trisha\Downloads\New folder\ANACONDA\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


[✓] Batch 1: Normal Activity
[✓] Batch 2: Normal Activity
[!] Batch 3: Abnormal Activity Detected
📡 Abnormal activity published to AWS IoT
[✓] Batch 4: Normal Activity
[!] Batch 5: Abnormal Activity Detected
📡 Abnormal activity published to AWS IoT
[✓] Batch 6: Normal Activity
[✓] Batch 7: Normal Activity
[✓] Batch 8: Normal Activity
[✓] Batch 9: Normal Activity
[!] Batch 10: Abnormal Activity Detected
📡 Abnormal activity published to AWS IoT


C:\Users\Trisha\AppData\Local\Temp\ipykernel_3920\2777328261.py:82: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  sensor_data = pd.read_sql(query, conn)
C:\Users\Trisha\Downloads\New folder\ANACONDA\Lib\site-packages\sklearn\base.py:493: UserWarning: X does not have valid feature names, but MinMaxScaler was fitted with feature names
  warnings.warn(


⚠️ Not enough data for sequence generation.


In [7]:
import boto3

# Initialize AWS IoT client
iot_client = boto3.client('iot', region_name='eu-north-1')

# Existing rule name
rule_name = "forwardDataRule"

# Define updated SQL statement and republish action
sql_statement = "SELECT * FROM 'serial/data'"
action = {
    "republish": {
        "roleArn": "arn:aws:iam::361769568387:role/IoTRepublishRole",  # Your existing IAM role
        "topic": "esp32/sub",
        "qos": 1
    }
}

# Replace the existing rule with updated payload
response = iot_client.replace_topic_rule(
    ruleName=rule_name,
    topicRulePayload={
        "sql": sql_statement,
        "actions": [action],
        "ruleDisabled": False
    }
)

print(f"Rule '{rule_name}' updated successfully.")


Rule 'forwardDataRule' updated successfully.
